In [1]:
using Pkg; Pkg.activate(dirname(pwd()))

  Activating project at `/media/yuehhua/Workbench/workspace/machine-learning.jl`


# K-Nearest Neighbors

In [2]:
using MLJ
using RDatasets

### Load data

In [3]:
smarket = dataset("ISLR", "Smarket")
first(smarket, 6)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…
1,2001.0,0.381,-0.192,-2.624,-1.055,5.01,1.1913,0.959,Up
2,2001.0,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
3,2001.0,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
4,2001.0,-0.623,1.032,0.959,0.381,-0.192,1.276,0.614,Up
5,2001.0,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
6,2001.0,0.213,0.614,-0.623,1.032,0.959,1.3491,1.392,Up


### Casting scientific types

In [4]:
y, X = unpack(smarket, ==(:Direction), colname -> true);
X = select(X, Not([:Year, :Today]))
first(X, 6)

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.381,-0.192,-2.624,-1.055,5.01,1.1913
2,0.959,0.381,-0.192,-2.624,-1.055,1.2965
3,1.032,0.959,0.381,-0.192,-2.624,1.4112
4,-0.623,1.032,0.959,0.381,-0.192,1.276
5,0.614,-0.623,1.032,0.959,0.381,1.2057
6,0.213,0.614,-0.623,1.032,0.959,1.3491


In [5]:
y = coerce(y, OrderedFactor)
classes(y[1])

2-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "Down"
 "Up"

### Training/testing set

In [6]:
train, test = partition(eachindex(y), 0.7, shuffle=true)

([466, 1177, 780, 71, 948, 141, 969, 500, 1237, 47  …  954, 1056, 735, 1072, 620, 889, 686, 575, 1236, 646], [148, 128, 513, 503, 619, 826, 720, 117, 632, 674  …  962, 158, 27, 367, 1149, 386, 781, 490, 474, 1122])

### Model

In [7]:
KNNClassifier = @load KNNClassifier pkg=NearestNeighborModels

import NearestNeighborModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/yuehhua/.julia/packages/MLJModels/lDzCR/src/loading.jl:168


NearestNeighborModels.KNNClassifier

In [8]:
knn = machine(KNNClassifier(K=3), X, y)

Machine trained 0 times; caches data
  model: KNNClassifier(K = 3, …)
  args: 
    1:	Source @399 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @741 ⏎ `AbstractVector{OrderedFactor{2}}`


### Training

In [9]:
fit!(knn, rows=train)

┌ Info: Training machine(KNNClassifier(K = 3, …), …).
└ @ MLJBase /home/yuehhua/.julia/packages/MLJBase/rQDaq/src/machines.jl:487


Machine trained 1 time; caches data
  model: KNNClassifier(K = 3, …)
  args: 
    1:	Source @399 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @741 ⏎ `AbstractVector{OrderedFactor{2}}`


### Predict

In [10]:
ŷ = predict_mode(knn, rows=test)

375-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "Down"
 "Up"
 "Down"
 "Down"
 "Up"
 "Down"
 "Down"
 "Down"
 "Down"
 "Down"
 "Down"
 "Down"
 "Up"
 ⋮
 "Up"
 "Up"
 "Up"
 "Down"
 "Up"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Down"
 "Down"

### Evaluation

In [11]:
accuracy(ŷ, y[test])

0.504